# Import libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import collections
import re
import string
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

nltk and gensim are two famous libraries that are used in Natural Language Processing (NLP). nltk library has been used to get the stop words of English language and to lemmatize words. Also gensim library has been used to load the Doc2vec model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
proj_dir='/content/drive/MyDrive/Colab Notebooks/doc2vec/' #give your project directory here. data sets should be in this location

# Load and clean the data

before converting into vectors text should be cleaned.

* Lower the english latters
* Remove headers
* Drop all digits
* Remove URLs and Emails
* Drop all punctuation from our text
* Drop stop words
* Lemmatize words


first, we convert all the letters into lowercase letters in order to avoid duplicating the same word ('Dog' and 'dog' are the same words. but the program will take these as two words. Hence we convert 'Dog' as 'dog' by doing letters lowercase)

After that we drop all the headers and numbers.Because headers are unwanted data and numbers don't have any meaning when we take them as a single word.

if any word contains '@','.com' or 'http' we drop those words. also we remove all punctuations and stop words. The words which are generally filtered out before processing a natural language are called stop words. These are actually the most common words such as “the”, “a”, “an”, “so”, “what” in English language. By removing these words, we remove the low-level information from our text in order to give more focus to the important information.

finally we lemmatize words. Lemmatization is the process wherein the context is used to convert a word to its meaningful base or root form. ex:- driving--> drive , dogs-->dog. To perform Lemmatization with Natural Language Tool Kit (NLTK), “WordNetLemmatizer()” method has been used

In [ ]:
'''this function is used to read text files.'''
def read_txt_file(file_name):
    with open(file_name,encoding="utf8", errors='ignore') as f:
        ###extract the boady of the text###
        line = f.readline()
        txt=''
        txt=txt+' '+line
        while line:
            line = f.readline()
            txt=txt+' '+line
        ###################################
    f.close()
    return(txt)

In [ ]:
stop_words=set(stopwords.words('english'))#load stop words
punctuations=string.punctuation #get punctuations
lemmatizer = WordNetLemmatizer()
'''this function is used to clean text'''
def clean_txt(txt):
    txt=txt.lower() #set all characters to lowercase
    sentences=txt.split('\n')
    txt = ' '.join([i for i in sentences if not ':' in i])#remove headers
    txt = ''.join([i for i in txt if not i.isdigit()])#remove numbers

    ###remove urls and emails###
    words=txt.split()
    txt = ' '.join([i for i in words if not '@' in i and not '.com' in i and not  'http:' in i])
    #######################################

    ###remove punctuations###
    for character in punctuations:
        txt = txt.replace(character, '')
    #########################################
    
    ###remove stop words and lemmatize###
    words=txt.split()
    filtered_txt = ' '.join([lemmatizer.lemmatize(i) for i in words if not i in stop_words])
    #####################################
    
    return(filtered_txt)

In [ ]:
def load_and_clean_data(location):    
    y=os.listdir(location)#get the list of folder
    txts=[]
    txts_cleaned=[]
    folder_array=[]
    file_array=[]
    for i in range(len(y)):
        text_file_names=os.listdir(location+'/'+y[i]) #get the list of files
        for text_file_name in text_file_names:
                file_array.append(text_file_name)
                txt=read_txt_file(location+'/'+y[i]+'/'+text_file_name) #read the text file
                txts.append(txt)
                txts_cleaned.append(clean_txt(txt)) #clean the text
                folder_array.append(y[i])

    ###create a data frame###
    df=pd.DataFrame()
    df['texts']=txts
    df['text cleaned']=txts_cleaned
    df['folder name']=folder_array
    df['file name']=file_array
    ########################
    return (df)



In [ ]:
df_train=load_and_clean_data(proj_dir+'20news-bydate-train')
df_test=load_and_clean_data(proj_dir+'20news-bydate-test')

In [ ]:
df_train.head()

,texts,text cleaned,folder name,file name
0,Subject: MIT R5 on Sun with Rasterops TC Colo...,possible run mit r based xserver sun rasterops...,comp.windows.x,66939
1,Subject: Automated X testing\n From: mark@tri...,anyone know available term automated testing x...,comp.windows.x,67117
2,Subject: Re: Trouble compiling X11R5 on SunOS...,ive trying compile xr patchlevel sun sparc ipx...,comp.windows.x,66947
3,"Organization: Ministry of Education, Computer...",hp workstation us pseudocolor id x color defau...,comp.windows.x,67279
4,To: gnu-gdb-bug@gatech.edu\n Distribution: wo...,hi trying write xwindows based interface run t...,comp.windows.x,67410


In [ ]:
df_test.head()

,texts,text cleaned,folder name,file name
0,Organization: University of Central Florida -...,hello considering buying hummingbird xwindows ...,comp.windows.x,67551
1,Organization: Queen's University at Kingston\...,tn program support xterm dont like x cant copy...,comp.windows.x,67566
2,Subject: Re: X on DOS or Windows\n From: clee...,im posting request since last one title im loo...,comp.windows.x,68196
3,From: iop@server2.iie.ncku.edu.tw\n Subject: ...,studying book unix desktop guide open look exa...,comp.windows.x,68332.eml
4,Subject: Re: Resource/Widget toolkit required...,anyone know public domain toolkits creating xw...,comp.windows.x,67972


# Convert to vectors

document is tokenized into words and applied doc2vec model. doc2vec is a method to represent list of words using a vector. it is used to create a vectorised representation of a group of words taken collectively as a single unit.In gensim the model will always be trained on a word per word basis. Therefore  we split the document into an array of words using split(). In order to train the model, tagged documents are needed. it can be created by using models.doc2vec.TaggedDcument(). then finally we train the doc2vec model

In [ ]:
'''this function is used to do tokenization'''
def tokenizer(txt):  
    tokens=txt.split(' ')
    unique_tokens=np.unique(np.array(tokens)) #get unique tokens
    ###create a dictonary of tokens###
    tokens_dict={}
    for indx in range(len(unique_tokens)):
        tokens_dict[unique_tokens[indx]]=indx
    return(tokens_dict,tokens)

In [ ]:
#tokanize train and test data
words_list=[]
for i in range(len(df_train)):
    _,words=tokenizer(df_train['text cleaned'][i]) 
    words_list.append(words)
for i in range(len(df_test)):
    _,words=tokenizer(df_test['text cleaned'][i])
    words_list.append(words)
####################################

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(words_list)]
model = Doc2Vec(vector_size=40, min_count=2, epochs=30) #fit the Doc2Vec model
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
###get train vectors##
train_vectors=[]
for i in range(len(df_train)):
    train_vectors.append(model.infer_vector(list(df_train['text cleaned'][i].split(' '))))
##########################

In [ ]:
##get test vectors##
test_vectors=[]
for i in range(len(df_test)):
    test_vectors.append(model.infer_vector(list(df_test['text cleaned'][i].split(' '))))
############################

In [ ]:
train=pd.DataFrame(train_vectors)
train['folder name']=df_train['folder name']
train['file name']=df_train['file name']
test=pd.DataFrame(test_vectors)
test['folder name']=df_test['folder name']
test['file name']=df_test['file name']

In [ ]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,folder name,file name
0,-0.066713,-0.287429,-0.399868,-0.333415,0.678753,0.437295,0.678954,0.788509,-0.330253,-0.016641,...,-0.806411,0.039909,0.491255,-0.058101,0.683467,-0.010286,-0.175499,0.299546,comp.windows.x,66939
1,0.744486,-0.531116,-1.070442,0.013823,0.180013,0.090380,0.437424,0.166477,0.301941,1.067428,...,-0.697233,0.104386,0.386378,0.043756,0.127149,-1.153150,-0.997933,-0.924932,comp.windows.x,67117
2,0.328144,-0.481195,-0.414951,-1.017945,-0.239436,0.405413,0.776452,0.451591,-0.740614,0.047171,...,-0.366408,0.547060,-0.487406,-0.095528,0.504356,0.231408,-0.145500,-0.685314,comp.windows.x,66947
3,0.131527,-0.154112,-0.048198,-0.397870,0.119279,-0.017622,0.029412,0.679335,-0.417777,0.563634,...,0.101432,0.175480,-0.124023,-0.221537,-0.062515,-0.325504,-0.053227,-0.203337,comp.windows.x,67279
4,-0.211842,0.199410,-1.498385,-0.261526,0.882079,-0.006605,-0.408872,1.374538,0.044579,0.548005,...,-0.936486,-0.546783,1.274282,-0.635842,-0.710933,-0.932430,-1.865840,-0.904930,comp.windows.x,67410


In [ ]:
test.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,folder name,file name
0,0.087660,-0.432065,-0.653577,-0.502173,0.643440,0.222418,-0.029119,0.177928,0.100904,0.031590,...,-0.616315,-0.121021,0.246841,0.034125,0.122506,-0.584005,-0.321968,-0.024909,comp.windows.x,67551
1,0.520183,0.181805,-0.273885,-0.156340,0.083651,-0.136866,-0.053114,0.256988,0.121570,0.544195,...,-0.135414,0.350382,-0.117181,-0.294207,-0.331065,-0.209273,-0.385534,-0.271347,comp.windows.x,67566
2,0.607010,-0.206781,0.280009,-1.150236,-0.072557,0.138987,0.024901,-0.050259,-0.060715,1.663651,...,-0.510634,0.125493,-0.556234,0.062671,0.462736,-0.808190,-1.024320,-0.719526,comp.windows.x,68196
3,-3.872827,1.148111,-0.683765,-0.862673,0.301422,6.109645,-0.048783,6.050083,0.046541,0.285950,...,-2.981766,2.518252,0.438007,1.758084,2.462524,0.004354,-5.147213,-1.866775,comp.windows.x,68332.eml
4,-0.382234,0.994797,-1.905553,0.269059,-0.128935,0.054541,-0.743769,-0.051312,-0.034497,1.555881,...,-0.685315,0.170255,-1.074199,-0.382189,-0.221441,-1.426607,-1.337920,-0.806584,comp.windows.x,67972


In [ ]:
train_,validation=train_test_split( train, test_size=0.33, random_state=42) #split data into train and validation sets

In [ ]:
len(train_),len(test),len(validation)

(7580, 7537, 3734)

In [ ]:
#save data sets
train_.to_csv(proj_dir+'train_data.csv')
test.to_csv(proj_dir+'test_data.csv')
validation.to_csv(proj_dir+'validation_data.csv')

# Modeling

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras import Sequential
from keras.utils.vis_utils import plot_model

## Load data

In [ ]:
train=pd.read_csv(proj_dir+'train_data.csv')
test=pd.read_csv(proj_dir+'test_data.csv')
validation=pd.read_csv(proj_dir+'validation_data.csv')

## Data rearrange for modeling

In [ ]:
X_train=train[[str(i) for i in range(40)]]
y_train=train['folder name']

X_test=test[[str(i) for i in range(40)]]
y_test=test['folder name']

X_validation=validation[[str(i) for i in range(40)]]
y_validation=validation['folder name']

In [ ]:
names=np.unique(np.array(y_train))

In [ ]:
y_train_en=[np.where(names==name)[0] for name in y_train ]
y_test_en=[np.where(names==name)[0] for name in y_test ]
y_validation_en=[np.where(names==name)[0] for name in y_validation ]

In [ ]:
y_train_en=keras.utils.to_categorical(y_train_en)
y_test_en=keras.utils.to_categorical(y_test_en)
y_validation_en=keras.utils.to_categorical(y_validation_en)